Using Tensorflow for Deeplearning and Tensorboard to illustrate and save our data models

In [47]:
from tensorflow.keras.callbacks import TensorBoard

In [48]:
from datetime import datetime

The log directory indicates where our data models will be saved.

In [3]:
log_directory = 'logs/fit'

In [4]:
timestamp = datetime.now().strftime("%Y-%m-%d--%H:%M")

In [5]:
log_directory = log_directory + '/' +timestamp
log_directory

'logs/fit/2020-04-29--15:18'

In [45]:
board = TensorBoard(log_dir = log_directory, 
                   histogram_freq=1,
                   write_graph = True,
                   write_images = True,
                   update_freq = "epoch",
                   )

In [8]:
import pandas as pd
df = pd.read_csv("bank_note_data.csv")

In [49]:
df[df.isnull()==False]

,Image.Var,Image.Skew,Image.Curt,Entropy,Class
0,3.62160,8.66610,-2.8073,-0.44699,0
1,4.54590,8.16740,-2.4586,-1.46210,0
2,3.86600,-2.63830,1.9242,0.10645,0
3,3.45660,9.52280,-4.0112,-3.59440,0
4,0.32924,-4.45520,4.5718,-0.98880,0
...,...,...,...,...,...
1367,0.40614,1.34920,-1.4501,-0.55949,1
1368,-1.38870,-4.87730,6.4774,0.34179,1
1369,-3.75030,-13.45860,17.5932,-2.77710,1
1370,-3.56370,-8.38270,12.3930,-1.28230,1


Partitioning the data and transforming the data (data should ideally be scaled, but since all the numbers are small, it's not necessary)

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
x_train, x_test, y_train, y_test = train_test_split(df.drop('Class', axis =1), df['Class'], test_size = 0.3)

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.preprocessing import MinMaxScaler

In [13]:
scaler = MinMaxScaler()

In [14]:
x_train = scaler.fit_transform(x_train)

In [15]:
x_test = scaler.transform(x_test)

Adding Dropout Layers and Early stopping in order to prevent overfitting 

In [16]:
model = Sequential()
model.add(Dense(30, activation = "relu"))
model.add(Dropout(0.2))
model.add(Dense(15, activation = "relu"))
model.add(Dropout(0.2))
model.add(Dense(1, activation = "sigmoid"))

In [17]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.


In [18]:
from tensorflow.keras.callbacks import EarlyStopping

In [19]:
early_stop = EarlyStopping(monitor = "val_loss", mode = "min", verbose = 1, patience = 25)

In [46]:
model.fit(x_train, y_train.values.reshape(960,1), epochs = 600, validation_data = (x_test, y_test.values.reshape(412,1)), verbose =1 ,callbacks=[early_stop, board])

Train on 960 samples, validate on 412 samples
Epoch 1/600
960/960 [==============================] - 0s 170us/sample - loss: 0.6817 - acc: 0.5708 - val_loss: 0.6747 - val_acc: 0.6578
Epoch 2/600
960/960 [==============================] - 0s 77us/sample - loss: 0.6640 - acc: 0.6854 - val_loss: 0.6591 - val_acc: 0.7136
Epoch 3/600
960/960 [==============================] - 0s 66us/sample - loss: 0.6442 - acc: 0.7417 - val_loss: 0.6376 - val_acc: 0.7330
Epoch 4/600
960/960 [==============================] - 0s 67us/sample - loss: 0.6172 - acc: 0.7750 - val_loss: 0.6090 - val_acc: 0.7816
Epoch 5/600
960/960 [==============================] - 0s 63us/sample - loss: 0.5831 - acc: 0.7740 - val_loss: 0.5713 - val_acc: 0.7694
Epoch 6/600
960/960 [==============================] - 0s 77us/sample - loss: 0.5406 - acc: 0.8042 - val_loss: 0.5295 - val_acc: 0.7791
Epoch 7/600
960/960 [==============================] - 0s 66us/sample - loss: 0.4928 - acc: 0.8073 - val_loss: 0.4844 - val_acc: 0.8083
E

Graphs and models are saved within the logs/fit folder. We can also predict values and test the accuracy of our neural network.

In [50]:
from sklearn.metrics import classification_report, confusion_matrix

In [51]:
predictions = model.predict_classes(x_test)
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))

[[230   0]
 [  0 182]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       230
           1       1.00      1.00      1.00       182

    accuracy                           1.00       412
   macro avg       1.00      1.00      1.00       412
weighted avg       1.00      1.00      1.00       412



Our neural network worked perfectly, garnering an accuracy rate of 100%.